In [1]:
import os, pickle, logging, pickle, joblib, sys, warnings
warnings.simplefilter('ignore')
from scipy import stats
import numpy as np
import pandas as pd

import catboost as cb

from sklearn import ensemble, metrics, pipeline, preprocessing, impute, model_selection
from scipy.stats import pearsonr, spearmanr

ML_RAW_PATH = "/data2/zhoujb/project/cowpea_project/basedXPXLR/ML/rawData/"
FS_PATH = "/data2/zhoujb/project/cowpea_project/basedXPXLR/ML/fs_PL/"
TEST_RES_PATH = "/data2/zhoujb/project/cowpea_project/basedXPXLR/ML/tesRes/"

In [2]:
feat_col = []
with open(os.path.join(FS_PATH, "cb_rfa_cv_pl_rmse")) as f:
    for line in f:
        feat_col.append(line.strip())

raw_data = pd.read_table(os.path.join(ML_RAW_PATH, "raw_data_PL.txt"), sep="\t", index_col=0)

target_col = ['HZ-PL']
raw_data = raw_data.dropna(subset=target_col)

#cat_col_names_raw = ["heading_stage_d", "leaf_blast_rep1", "leaf_blast_rep2", "leaf_blast_average", "GR_D3", "GR_D2", "GR_D1"]
#cat_col_names_raw = ["feat_{}".format(x) for x in cat_col_names_raw]
#cat_col_names = [i for i in feat_col if i in cat_col_names_raw]
#num_col_names = [i for i in feat_col if (i.startswith("feat")) and (i not in cat_col_names)]
#cat_col_num = []
#for item in cat_col_names:
    #cat_col_num.append(feat_col.index(item))

#for col in cat_col_names:
    #raw_data[col] = raw_data[col].astype("str")

kf = model_selection.KFold(n_splits=5, shuffle=True,  random_state=0)
y_test_final, y_pred_final = [], []
for i, (train_index, test_index) in enumerate(kf.split(raw_data)):
    data_train = raw_data.iloc[train_index].copy()
    data_test = raw_data.iloc[test_index].copy()

    scale_tool = preprocessing.StandardScaler()
    scale_tool.fit(data_train.loc[:, feat_col])
    data_train.loc[:, feat_col] = scale_tool.transform(data_train.loc[:, feat_col])
    data_test.loc[:, feat_col] = scale_tool.transform(data_test.loc[:, feat_col])

    train_sel = data_train.sample(frac=0.8, random_state=0)
    val_sel = data_train.drop(train_sel.index).copy()

    X_train = train_sel[feat_col].copy()
    y_train = train_sel[target_col].values.ravel()

    X_val = val_sel[feat_col].copy()
    y_val = val_sel[target_col].values.ravel()

    X_test = data_test[feat_col].copy()
    y_test = data_test[target_col].values.ravel()

    # Initialize CatBoostClassifier
    clf_model = cb.CatBoostRegressor(random_state=0, thread_count=4, loss_function='RMSE')
    # Fit model
    clf_model.fit(X_train, y_train, eval_set=[(X_train, y_train), (X_val, y_val)], verbose=1, plot=False)

    y_pred = clf_model.predict(X_test)

    y_test_final.append(y_test)
    y_pred_final.append(y_pred)

Learning rate set to 0.039298
0:	learn: 16.0972043	test: 16.0972043	test1: 16.2764979	best: 16.2764979 (0)	total: 51.4ms	remaining: 51.4s
1:	learn: 15.8230977	test: 15.8230977	test1: 16.0049214	best: 16.0049214 (1)	total: 52.6ms	remaining: 26.2s
2:	learn: 15.5216184	test: 15.5216184	test1: 15.7441023	best: 15.7441023 (2)	total: 53.6ms	remaining: 17.8s
3:	learn: 15.2402857	test: 15.2402857	test1: 15.4970995	best: 15.4970995 (3)	total: 54.6ms	remaining: 13.6s
4:	learn: 14.9472212	test: 14.9472212	test1: 15.2503667	best: 15.2503667 (4)	total: 55.6ms	remaining: 11.1s
5:	learn: 14.6544129	test: 14.6544129	test1: 14.9747092	best: 14.9747092 (5)	total: 56.6ms	remaining: 9.37s
6:	learn: 14.4324165	test: 14.4324165	test1: 14.7867041	best: 14.7867041 (6)	total: 57.3ms	remaining: 8.13s
7:	learn: 14.2033218	test: 14.2033218	test1: 14.5792750	best: 14.5792750 (7)	total: 58.3ms	remaining: 7.23s
8:	learn: 13.9597681	test: 13.9597681	test1: 14.3280866	best: 14.3280866 (8)	total: 59.2ms	remaining: 6.52

In [3]:
res_df = pd.DataFrame(columns=["Model", "Times", "Score", "Type"])
for i in range(len(y_test_final)):
    ## Get score
    #score_pear = pearsonr(y_test_final[i], y_pred_final[i])[0]
    score_spear = spearmanr(y_test_final[i], y_pred_final[i])[0]
    #score_rmse = metrics.mean_squared_error(y_test_final[i], y_pred_final[i], squared=False)
    score_rmse = metrics.root_mean_squared_error(y_test_final[i], y_pred_final[i])
    score_nrmse = score_rmse / np.std(y_test_final[i])

    #res_df.loc[len(res_df)] = ["CB_RFA", i+1, score_pear, "R"]
    res_df.loc[len(res_df)] = ["CB_RFA", i+1, score_spear, "R"]
    res_df.loc[len(res_df)] = ["CB_RFA", i+1, score_rmse, "RMSE"]
    res_df.loc[len(res_df)] = ["CB_RFA", i+1, score_nrmse, "NRMSE"]

with open(os.path.join(TEST_RES_PATH, "PL_CB_RFA.pickle"), "wb") as out_f:
    pickle.dump(res_df, out_f)

In [2]:
with open(os.path.join(TEST_RES_PATH, "PL_CB_RFA.pickle"), "rb") as f:
    res_df = pickle.load(f)

res_df

,Model,Times,Score,Type
0,CB_RFA,1,0.911998,R
1,CB_RFA,1,6.255264,RMSE
2,CB_RFA,1,0.375866,NRMSE
3,CB_RFA,2,0.912607,R
4,CB_RFA,2,7.743412,RMSE
5,CB_RFA,2,0.434356,NRMSE
6,CB_RFA,3,0.901526,R
7,CB_RFA,3,7.277973,RMSE
8,CB_RFA,3,0.447063,NRMSE
9,CB_RFA,4,0.862032,R


In [3]:
res_df.groupby(["Type"])["Score"].mean()

Type
NRMSE    0.428765
R        0.891271
RMSE     6.946691
Name: Score, dtype: float64

In [ ]:
0.891